In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
##Load model
model = load_model('model.h5')

In [3]:
## Load pickles
with open('one_hot_encoder_geo.pkl', "rb") as file:
    one_hot_encoder_geo = pickle.load(file)

with open('gender_encoder.pkl', "rb") as file:
    gender_encoder = pickle.load(file)

with open('scalar.pkl', "rb") as file:
    scalar = pickle.load(file)

In [4]:
# Example data
ex_data = {
    "CreditScore": 600,
    "Geography" : "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [5]:
geo_encoded = one_hot_encoder_geo.transform(np.array(ex_data["Geography"]).reshape(-1,1)).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = ["France", "Germany", "Spain"])
geo_encoded_df

,France,Germany,Spain
0,1.0,0.0,0.0


In [6]:
input_data = pd.concat([pd.DataFrame([ex_data]), geo_encoded_df], axis=1)

In [7]:
input_data.drop(columns=["Geography"], inplace=True)

In [8]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_data["Gender"] = gender_encoder.transform(input_data["Gender"])

In [10]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
##Scaling
input_data = scalar.transform(input_data)

In [12]:
input_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

**Prediction**

In [13]:
model.predict(input_data)
pred_prob = model.predict(input_data)[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [14]:
if pred_prob > 0.5:
    print("The customer is likely to leave the bank")
else:
    print("The customer is unlikely to leave the bank")

The customer is unlikely to leave the bank
